In [54]:
import pandas as pd
from sklearn.cluster import MeanShift
import numpy as np
from collections import Counter

In [47]:
clustering = MeanShift()
df = pd.read_csv("./umn_foursquare_datasets/checkins.dat", sep='|',skiprows=[1])
df = df.loc[df[u'     latitude      '].notnull() & df['     longitude     '].notnull(),].loc[:100000,:]


df=df.rename(columns={col:col.strip() for col in df.columns})

df['latitude'] = df['latitude'].apply(lambda x: x.strip())
df['longitude'] = df['longitude'].apply(lambda x: x.strip())

df=df[~ (df[[u'latitude', u'longitude']]=='').any(1)]

# data = pd.DataFrame(
#     {
#         "lat":df['latitude'],#.astype(np.float),
#         "lon":df['longitude']#.apply(np.float)
#     }
# )

df["latitude"] = df["latitude"].apply(pd.to_numeric, errors='ignore')
df["longitude"] = df["longitude"].apply(pd.to_numeric, errors='ignore')

In [48]:
df

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18
11,984483,1030290.0,955969.0,32.221743,-110.926479,2012-04-21 17:58:54
12,984685,304253.0,23558.0,40.650000,-73.950000,2012-04-21 18:19:34
13,984470,720850.0,749715.0,33.448377,-112.074037,2012-04-21 17:02:47
15,984610,1639666.0,442605.0,33.414768,-111.909309,2012-04-21 18:04:58
18,984653,1647192.0,23558.0,42.358431,-71.059773,2012-04-21 18:23:22


In [49]:
clustering.fit(df[[u'latitude', u'longitude']])

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [50]:
clustering.cluster_centers_

array([[  39.15243019,  -79.9229138 ],
       [  35.66751357, -115.26468594],
       [  50.09392785,    3.8505934 ],
       [  -5.06000002,  107.35464047],
       [ -24.55209123,  -47.82273966],
       [  27.67179617,  121.45579378],
       [  -0.34355106,   36.5556421 ],
       [  21.10643325, -157.50709945],
       [  15.13021347,  -68.47056876],
       [  29.30808302,   42.20855989],
       [ -34.45745856,  149.40721106],
       [  61.27960529, -148.89936404],
       [ -33.72367054,  -62.87812805],
       [  -0.88880902,  -77.91388456],
       [  19.34735175,   79.36585508],
       [ -30.77720589,   24.59628067],
       [ -38.41638574,  174.01883603],
       [  55.75927172,   90.39335887],
       [ -31.66687253,  117.27785207],
       [   6.3625    ,    2.4255    ],
       [  63.9126321 ,  -22.1916474 ],
       [  56.837814  ,   60.596842  ],
       [  -3.7183943 ,  -38.5433948 ]])

In [55]:
ctr=Counter(clustering.labels_)

In [58]:
ctr.values()

[25834,
 22553,
 488,
 460,
 123,
 149,
 94,
 94,
 54,
 81,
 39,
 41,
 34,
 35,
 23,
 9,
 8,
 4,
 4,
 1,
 1,
 29,
 1]

In [62]:
result = filter(lambda x: x[0]>15, zip(ctr.values(), clustering.cluster_centers_))

In [69]:
result

[(25834, array([ 39.15243019, -79.9229138 ])),
 (22553, array([  35.66751357, -115.26468594])),
 (488, array([50.09392785,  3.8505934 ])),
 (460, array([ -5.06000002, 107.35464047])),
 (123, array([-24.55209123, -47.82273966])),
 (149, array([ 27.67179617, 121.45579378])),
 (94, array([-0.34355106, 36.5556421 ])),
 (94, array([  21.10643325, -157.50709945])),
 (54, array([ 15.13021347, -68.47056876])),
 (81, array([29.30808302, 42.20855989])),
 (39, array([-34.45745856, 149.40721106])),
 (41, array([  61.27960529, -148.89936404])),
 (34, array([-33.72367054, -62.87812805])),
 (35, array([ -0.88880902, -77.91388456])),
 (23, array([19.34735175, 79.36585508])),
 (29, array([56.837814, 60.596842]))]

In [66]:
# with open("result.txt",'w') as fout:
#     for item in result:
#         fout.write(str(item[1][0]) + " " + str(item[1][0])+"\n")

In [68]:
offices = [
    ("Los Angeles", (33.751277, -118.188740)),
    ("Miami", (25.867736, -80.324116)),
    ("London", (51.503016, -0.075479)),
    ("Amsterdam", (52.378894, 4.885084)),
    ("Beijing", (39.366487, 117.036146)),
    ("Sydney", (-33.868457, 151.205134))
]

In [72]:
min_dist=None
min_clust=None
for res in result:
    for of in offices:
        if not min_dist:
            min_dist = (res[1][0]-of[1][0])**2+(res[1][1]-of[1][1])**2
            min_clust = res[1]
        else:
            dist = (res[1][0]-of[1][0])**2+(res[1][1]-of[1][1])**2
            if dist<min_dist:
                min_dist = dist
                min_clust = res[1]

In [73]:
min_clust

array([-34.45745856, 149.40721106])

In [74]:
with open("result.txt",'w') as fout:
    fout.write("-34.45745856 149.40721106")